In [1]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from transformers import RobertaTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

nltk.download('stopwords')
nltk.download('punkt')

/Users/rodolfocacacho/miniforge3/envs/lsd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
torch.cuda.is_available()

False

In [3]:
file_train = 'data/subtask1/train.json'
file_validation = 'data/subtask1/validation.json'

# Open the JSON file
with open(file, 'r') as file:
    # Load the JSON data
    data = json.load(file)

# Create a list of dictionaries
entries_list = []

# Iterate through each entry in the JSON data
for entry in data:
    # Extract relevant information from each entry
    entry_dict = {
        "id": entry["id"],
        "text": entry["text"],
        "labels": entry["labels"],
        "link": entry["link"]
    }

    # Append the dictionary to the list
    entries_list.append(entry_dict)

# Now 'entries_list' contains a list of dictionaries, each representing an entry in your JSON file
# You can access the information as needed
for entry_dict in entries_list:
    print(f"ID: {entry_dict['id']}, Text: {entry_dict['text']}, Labels: {entry_dict['labels']}, Link: {entry_dict['link']}")


NameError: name 'file' is not defined

In [ ]:
# Create a new list to store the preprocessed dictionaries
preprocessed_list = []

for entry in entries_list:
    preprocessed_entry = entry.copy()
    preprocessed_entry['text'] = preprocess_text(entry['text'])
    if len(preprocessed_entry['labels']) == 0:
        preprocessed_entry['labels'] = ['nocat']
    else:
        t = preprocessed_entry.get('labels',[0])[0]
        # print(f'aos {t}')
        preprocessed_entry['labels'] = [t]
        # preprocessed_entry['labels'] = preprocessed_entry['labels']    
    preprocessed_list.append(preprocessed_entry)


# Display the preprocessed data
for id,entry in enumerate(preprocessed_list):
    print(f"ID: {entry['id']}, Text: {entry['text']} Label: {entry['labels']}")
    # print(f"ID: {entries_list[id]['id']}, Text: {entries_list[id]['text']}")

In [4]:

# Initialize a dictionary to store label counts per category
category_counts = {}
ids_ohne_label = []
ids_mit_label = []
max_id = 0

# Loop through the dataset
for entry in preprocessed_list:
    labels = entry.get('labels', [])  # Get the labels for the current entry

    if not labels:
        ids_ohne_label.append(entry['id'])
    else:
        ids_mit_label.append(entry['id'])
        for label in labels:
            if label not in category_counts:
                category_counts[label] = 1
            else:
                category_counts[label] += 1
    if len(labels) > max_id:
        max_id = len(labels)
        idx = entry['id']


# Display the label counts per category
for category, count in category_counts.items():
    print(f"Category: {category}, Count: {count}")

f_list = [d for d in preprocessed_list if d.get('id') == idx]

print(f'ids ohne labels: {len(ids_ohne_label)} ids mit labels: {len(ids_mit_label)} max_labels {max_id}')
print(f'id {idx} text: {f_list}')

NameError: name 'preprocessed_list' is not defined

In [5]:
# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text for each entry in the dataset
for entry in preprocessed_list:
    text = entry['text']
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Update the entry with tokenized information
    entry['input_ids'] = tokens['input_ids']
    entry['attention_mask'] = tokens['attention_mask']

# Display the tokenized entries
for entry in preprocessed_list:
    print(f"ID: {entry['id']}")
    print(f"Text: {entry['text']}")
    print("Token IDs:", entry['input_ids'])
    print("Attention Mask:", entry['attention_mask'])
    print()

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

c:\ProgramData\miniconda3\envs\python3.9\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\meikf\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

NameError: name 'preprocessed_list' is not defined

In [3]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Replace multiple whitespace characters with a single space
    text = re.sub(r'\s+', ' ', text)
    # Replace line breaks with a space
    text = text.replace('\\n', ' ')
    text = text.replace('\n', ' ')

    # Replace dashes with spaces
    text = text.replace('-', ' ')

    # Remove special characters, punctuation (except apostrophes), and symbols
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if token not in stop_words]
    
    # Porter stemming (optional)
    # stemmer = PorterStemmer()
    # tokens = [stemmer.stem(token) for token in tokens]
    
    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def read_json_labels(file):
    # Open the JSON file
    with open(file, 'r') as file:
        # Load the JSON data
        data = json.load(file)

    # Create a list of dictionaries
    entries_list = []
    # Iterate through each entry in the JSON data
    for entry in data:
        # Extract relevant information from each entry
        entry_dict = {
            "id": entry["id"],
            "text": entry["text"],
            "labels": entry["labels"],
            "link": entry["link"]
        }

        # Append the dictionary to the list
        entries_list.append(entry_dict)
    
    return entries_list


def preprocess_text_list(entries_list):
    # Create a new list to store the preprocessed dictionaries
    preprocessed_list = []

    for entry in entries_list:
        preprocessed_entry = entry.copy()
        preprocessed_entry['text'] = preprocess_text(entry['text'])
        if len(preprocessed_entry['labels']) == 0:
            preprocessed_entry['labels'] = ['nocat']
        else:
            t = preprocessed_entry.get('labels',[0])[0]
            # print(f'aos {t}')
            preprocessed_entry['labels'] = [t]
            # preprocessed_entry['labels'] = preprocessed_entry['labels']    
        preprocessed_list.append(preprocessed_entry)
    
    return preprocessed_list

def preprocess_labels(entries_list):

    # Create a new list to store the preprocessed dictionaries
    preprocessed_list = []

    for entry in entries_list:
        preprocessed_entry = entry.copy()
        preprocessed_entry['text'] = preprocess_text(entry['text'])
        if len(preprocessed_entry['labels']) == 0:
            preprocessed_entry['labels'] = ['nocat']
        else:
            t = preprocessed_entry.get('labels',[0])[0]
            # print(f'aos {t}')
            preprocessed_entry['labels'] = [t]
            # preprocessed_entry['labels'] = preprocessed_entry['labels']    
        preprocessed_list.append(preprocessed_entry)

    return preprocessed_list

# def tokenize_text(preprocessed_list):
#     # Tokenize the text for each entry in the dataset
#     for entry in preprocessed_list:
#         text = entry['text']
#         tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

#         # Update the entry with tokenized information
#         entry['input_ids'] = tokens['input_ids']
#         entry['attention_mask'] = tokens['attention_mask']
    
#     return preprocessed_list

def tokenize_text(preprocessed_list, tokenizer):
    tokenized_list = []  # Create a new list to store tokenized entries

    # Tokenize the text for each entry in the dataset
    for entry in preprocessed_list:
        text = entry['text']
        tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

        # Create a new entry with tokenized information
        tokenized_entry = {
            'id': entry['id'],
            'input_ids': tokens['input_ids'],
            'attention_mask': tokens['attention_mask']
        }

        tokenized_list.append(tokenized_entry)

    return tokenized_list

def extract_lists(data_list):
    texts = []
    ids = []
    labels = []

    for entry in data_list:
        text = entry.get('text', '')
        id_value = entry.get('id', '')
        label_value = entry.get('labels', '')

        texts.append(text)
        ids.append(id_value)
        labels.extend(label_value)
        # labels.append(label_value)

    return texts, ids, labels




In [4]:
file_train = 'data/subtask1/train.json'
file_validation = 'data/subtask1/validation.json'
# Create or load a label encoder
label_encoder = LabelEncoder()

# Train
train_json = read_json_labels(file_train)
train_list = preprocess_text_list(train_json)
train_list = preprocess_labels(train_list)
train_text_list, train_id_list, train_labels_list = extract_lists(train_list)
train_encoded_labels = label_encoder.fit_transform(train_labels_list)
train_tokens = tokenizer(train_text_list, padding=True, truncation=True, return_tensors="pt")
train_tokens_input_ids = train_tokens['input_ids']
train_tokens_mask_ids = train_tokens['attention_mask']

# train_list = tokenize_text(train_list,tokenizer)

# # Validation
val_json = read_json_labels(file_validation)
val_list = preprocess_text_list(val_json)
val_list = preprocess_labels(val_list)
val_text_list, val_id_list, val_labels_list = extract_lists(val_list)
val_encoded_labels = label_encoder.transform(val_labels_list)
val_tokens = tokenizer(val_text_list, padding=True, truncation=True, return_tensors="pt")
val_tokens_input_ids = val_tokens['input_ids']
val_tokens_mask_ids = val_tokens['attention_mask']



In [12]:
# Create DataLoader
train_dataset = TensorDataset(train_tokens_input_ids, train_tokens_mask_ids, torch.tensor(train_encoded_labels,dtype=torch.long))
val_dataset = TensorDataset(val_tokens_input_ids, val_tokens_mask_ids, torch.tensor(val_encoded_labels,dtype=torch.long))

batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,shuffle=False)

# Load pre-trained RoBERTa model and tokenizer
model_name = 'roberta-base'
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))
# tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader))

# Loss Function
criterion = torch.nn.CrossEntropyLoss()



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/rodolfocacacho/miniforge3/envs/lsd/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to GPU
model = model.to(device)

# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        # Move batch tensors to the same device as the model
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {average_loss}")

 # Validation
    model.eval()
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            # print(f'logits {logits} size {logits.size()}')
            # val_preds = (torch.sigmoid(logits) > 0.5).long()
            val_preds = torch.argmax(logits, dim=1)
            # print(f'val_preds {val_preds} size {val_preds.size()}')
            
            all_val_labels.append(labels.numpy())
            all_val_preds.append(val_preds.numpy())


    # Flatten the list of numpy arrays
    all_val_labels = np.concatenate(all_val_labels)
    all_val_preds = np.concatenate(all_val_preds)

    # Convert to PyTorch tensors
    all_val_labels = torch.tensor(all_val_labels).float()
    all_val_preds = torch.tensor(all_val_preds).float()

    print(f'preds {all_val_preds} labels {all_val_labels}')
    # Calculate F1-score for validation set
    f1 = f1_score(all_val_labels, all_val_preds, average='micro',zero_division="warn")
    print(f'f1 {f1} size {f1.size}')
    print(f"Epoch {epoch + 1}/{epochs}, Validation F1-Score: {f1}")

Epoch 1/3, Average Training Loss: 3.0380727450052896
preds tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]) labels tensor([ 5.,  6., 18.,  8.,  0.,  9., 20., 17.,  7.,  0.,  0.,  3., 17.,  9.,
         0., 11., 17., 11.,  5.,  5.])
f1 0.0 size 1
Epoch 1/3, Validation F1-Score: 0.0
Epoch 2/3, Average Training Loss: 3.0482051372528076
preds tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]) labels tensor([ 5.,  6., 18.,  8.,  0.,  9., 20., 17.,  7.,  0.,  0.,  3., 17.,  9.,
         0., 11., 17., 11.,  5.,  5.])
f1 0.0 size 1
Epoch 2/3, Validation F1-Score: 0.0
Epoch 3/3, Average Training Loss: 3.033432881037394
preds tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]) labels tensor([ 5.,  6., 18.,  8.,  0.,  9., 20., 17.,  7.,  0.,  0.,  3., 17.,  9.,
         0., 11., 17., 11.,  5.,  5.])
f1 0.0 size 1
Epoch 3/3, Validation F1-Score: 0.0
